In [33]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from tensorflow.keras.layers import *
import cv2
import pickle
from scipy.ndimage import median_filter, convolve
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ls '/content/drive/My Drive/flowers/'

daisy/  dandelion/  flowers/  rose/  sunflower/  tulip/


In [5]:
data_dir = "/content/drive/My Drive/flowers"

categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [6]:
data = []
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224,224))
                
                image = np.array(image, dtype=np.float32)
                
                data.append([image, label])
            except Exception as e:
                pass
    print(len(data))
    
    pik = open('data.pickle', 'wb')
    pickle.dump(data,pik)
    pik.close()

In [7]:
make_data()

4346


In [8]:
def load_data():
    pick = open('data.pickle','rb')
    data = pickle.load(pick)
    pick.close()
    
    np.random.shuffle(data)
    
    feature = []
    labels = []
    
    for img, label in data:
        feature.append(img)
        labels.append(label)
        
    feature = np.array(feature, dtype=np.float32)
    labels = np.array(labels)
    
    feature = feature/255.0
    
    return [feature, labels]

In [ ]:
(feature, labels) = load_data()

In [1]:
labels

NameError: ignored

In [10]:

labels = labels.reshape((-1,1))

In [11]:

labels.shape

(4346, 1)

In [12]:


x_train, x_test, y_train, y_test = train_test_split (feature, labels, test_size = 0.1)

In [13]:
x_train.shape

(3911, 224, 224, 3)

In [51]:
y_test.shape

(435, 1)

In [45]:
input_shape = (224,224,3)
input = tf.keras.layers.Input(input_shape, dtype = tf.uint8)
x = tf.cast(input, tf.float32)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
#x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.5)(x)
#x = tf.keras.layers.experimental.preprocessing.RandomFlip()(x)

base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=input_shape,
    include_top=False)
base_model.trainable = False
x = base_model(input, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(5)(x)
output = tf.keras.layers.Softmax()(output)

model = tf.keras.Model(input, output)
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 6405      
_________________________________________________________________
softmax_3 (Softmax)          (None, 5)                 0         
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
________________________________________

In [46]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [25]:
batch_size = 64
epochs = 20
patience = 5
model.fit(x=x_train,y=y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), 
          callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience)])

Epoch 1/20
62/62 [==============================] - 5s 75ms/step - loss: 1.6630 - accuracy: 0.2659 - val_loss: 1.5101 - val_accuracy: 0.3517
Epoch 2/20
62/62 [==============================] - 4s 64ms/step - loss: 1.6151 - accuracy: 0.2928 - val_loss: 1.4819 - val_accuracy: 0.3632
Epoch 3/20
62/62 [==============================] - 4s 64ms/step - loss: 1.5584 - accuracy: 0.3086 - val_loss: 1.4584 - val_accuracy: 0.3816
Epoch 4/20
62/62 [==============================] - 4s 63ms/step - loss: 1.5279 - accuracy: 0.3283 - val_loss: 1.4638 - val_accuracy: 0.3862
Epoch 5/20
62/62 [==============================] - 4s 64ms/step - loss: 1.5188 - accuracy: 0.3296 - val_loss: 1.5012 - val_accuracy: 0.3816
Epoch 6/20
62/62 [==============================] - 4s 64ms/step - loss: 1.5045 - accuracy: 0.3454 - val_loss: 1.5412 - val_accuracy: 0.3034
Epoch 7/20
62/62 [==============================] - 4s 64ms/step - loss: 1.4962 - accuracy: 0.3426 - val_loss: 1.4546 - val_accuracy: 0.4184
Epoch 8/20
62

In [47]:
y_pred = model.predict(x_test)

In [48]:
y_pred.shape

(435, 5)

In [49]:
x_test.shape

(435, 224, 224, 3)

In [50]:
y_test.shape

(435, 1)

In [40]:
sklearn.metrics.confusion_matrix(y_test, y_pred)

ValueError: ignored